<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Teste_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotinas de Criação de Matriz de Embeddings


In [ ]:
! pip install keras==2.7.0
! pip show keras

     |████████████████████████████████| 1.3 MB 27.4 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.


Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: tensorflow, keras-vis


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy
import itertools
import nltk
import string, re
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import loadtxt
from numpy import savetxt
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence

In [3]:
########## PATHS DE ARQUIVOS ##########
# 
PATH_CBOW_s300 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s300_modifyed.txt"

#
PATH_BASE_1_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_1 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV" #saida
#
PATH_BASE_2_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_2 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV" #saida

In [4]:
#DataFrame_Base= pd.read_csv(PATH_BASE_1_CLASSIFICADA)
DataFrame_Base = pd.read_csv(PATH_BASE_2_CLASSIFICADA)
DataFrame_Base.head()

,Unnamed: 0,label,text
0,0,1,ambiguinho eu estou com nojo
1,1,1,menino do céu para que está feio
2,2,1,já pergunto a porra do trem já sabendo da vdd ...
3,3,1,sardella só faz merda
4,4,1,foda se na real vou tratar igual


In [28]:
palavras_unicas = list(DataFrame_Base['text'].str.split(' ', expand=True).stack().unique())
print(len(palavras_unicas))
# retira eventuais numeros. Numeros unicos por algum motivo matam o processo de filtragem. 
palavras_unicas = set(map(lambda x: re.findall(r'^\D+', x)[0] if re.findall(r'^\D+', x) != [] else "a" ,palavras_unicas))

for i in palavras_unicas:
  if len(i.split()) > 1:
    print(i)

9144


# Filtra os embeddings

In [6]:
# Código para filtrar o arquivo de embeddings.

embeddings_index = dict()

f = open(PATH_CBOW_s300, encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    if (word in palavras_unicas):
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

In [7]:
def monta_linha(palavra, coeficientes):
    # Monta a linha com palavra _ coeficientes.
    
    linha = f'{palavra}'
    for coef in coeficientes:
        linha += f' {coef:.5f}'
    linha += '\n'
    
    return linha

In [8]:
lista_de_embeddings = []
for word in embeddings_index:
    coeficientes = embeddings_index[word]
    linha = monta_linha(word, coeficientes)
    lista_de_embeddings.append(linha)

In [9]:
embeddings_dict = {}
for i, line in enumerate(lista_de_embeddings):
    values = line.split()
    embeddings_dict[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [10]:
token = text.Tokenizer()
token.fit_on_texts(palavras_unicas)
#word_index: dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.
word_index = token.word_index

In [18]:
# create token-embedding mapping

lista_de_chaves = list(embeddings_dict.keys()) #palavras

tamanho_da_dimensao = len(embeddings_dict[lista_de_chaves[0]])

embedding_matrix = numpy.zeros((len(word_index) + 1, tamanho_da_dimensao ))

for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
print(word_index)

{'br': 1, 'brasil': 2, 'pdt': 3, 'amanda': 4, 'm': 5, 'com': 6, 'sempre': 7, 'psol': 8, 'fogo': 9, 'ei': 10, 'nilson': 11, 'pt': 12, 'mc': 13, 'jandira': 14, 'feghali': 15, 'luciano': 16, 'alan': 17, 'pepper': 18, 'taxista': 19, 'atura': 20, 'cadeia': 21, 'grupos': 22, 'bolsominions': 23, 'batemos': 24, 'esfaqueamento': 25, 'regianebalbino': 26, 'hlaranja': 27, 'conseguia': 28, 'motivada': 29, 'dolorido': 30, 'luizcosta': 31, 'natalyfraga': 32, 'cube': 33, 'jeri': 34, 'análisenão': 35, 'jessibto': 36, 'marinacomciro': 37, 'recado': 38, 'está': 39, 'baba': 40, 'dizer': 41, 'seita': 42, 'total': 43, 'antibacteriano': 44, 'daciolo': 45, 'são': 46, 'devo': 47, 'energia': 48, 'defecar': 49, 'muliro': 50, 'nopau': 51, 'conselheiros': 52, 'festival': 53, 'gastronômica': 54, 'real': 55, 'criistiano': 56, 'consigo': 57, 'pitty': 58, 'mg': 59, 'verá': 60, 'camisas': 61, 'ao': 62, 'éramos': 63, 'querido': 64, 'notmoranatall': 65, 'gramática': 66, 'inhumas': 67, 'troquei': 68, 'estencão': 69, 'dep

In [21]:
print(type(embedding_matrix))
embedding_matrix.shape

<class 'numpy.ndarray'>


(9085, 300)

In [12]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.17693999 -0.04666    -0.148      ... -0.27713001  0.17879
   0.0385    ]
 [ 0.40718001 -0.28492001  0.00271    ... -0.27092999 -0.21365
   0.07886   ]
 ...
 [-0.49766999 -0.22409999  0.18630999 ... -0.14623     0.07545
  -0.23986   ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.38782001 -0.24257    -0.15741    ... -0.1143      0.1391
   0.02957   ]]


In [14]:
# save to csv file
savetxt(PATH_MATRIZ_CBOW_300_BASE_2, embedding_matrix, delimiter=',')

In [ ]:
# data = loadtxt('data.csv', delimiter=',')
# # print the array
# print(data)